Import all libraries


In [2]:
import os
import zipfile
import requests
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import wn
from wn.similarity import wup
from tqdm import tqdm
from nltk.corpus.reader.wordnet import WordNetCorpusReader
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from deep_translator import GoogleTranslator
from collections import defaultdict
from gensim.models import Word2Vec
import gensim
import gensim.downloader as api
import kagglehub
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import tarfile
from multiprocessing import Pool, Manager
import concurrent.futures
import pickle
import torch
from transformers import MarianMTModel, MarianTokenizer

In [ ]:
# Login to Kaggle
kagglehub.login()

In [ ]:
## Clone WNAffect repository
repo_url = "https://github.com/clemtoy/WNAffect.git"
clone_dir = "WNAffect"

if not os.path.exists(clone_dir):
    os.system(f"git clone {repo_url} {clone_dir}")
else:
    print("Failed to clone repository. Directory already exists.")

from WNAffect.wnaffect import WNAffect
from WNAffect.emotion import Emotion

# Download the nltk data
nltk.download("wordnet")
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download("omw-1.4")

# Download finnish wordnet
!python -m wn download omw-fi

# Download finnish word2vec model from kaggle
word2vec_fi = kagglehub.dataset_download("lehtol/word2vec-fi")

In [5]:
def download_and_extract(url, zip_path, extract_path):
    if os.path.exists(extract_path):
        print(f"File {extract_path} already exists.")
    else:
        print(f"Downloading and extracting the {zip_path} to {extract_path}.")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get("content-length", 0))
        block_size = 1024
        with open(zip_path, "wb") as file, tqdm(
            desc=zip_path,
            total=total_size,
            unit="iB",
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(block_size):
                file.write(data)
                bar.update(len(data))
        if not os.path.exists(extract_path):
            os.makedirs(extract_path)
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            for member in zip_ref.namelist():
                member_path = os.path.join(
                    extract_path,
                    os.path.relpath(
                        member, start=os.path.commonpath(zip_ref.namelist())
                    ),
                )
                if member.endswith("/"):
                    os.makedirs(member_path, exist_ok=True)
                else:
                    os.makedirs(os.path.dirname(member_path), exist_ok=True)
                    with open(member_path, "wb") as f:
                        f.write(zip_ref.read(member))
        os.remove(zip_path)
        print(f"Download and extraction of {extract_path} complete.")


def download(url, file_path):
    if os.path.exists(file_path):
        print(f"File {file_path} already exists.")
    else:
        print(f"Downloading the file {file_path}")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get("content-length", 0))
        block_size = 1024

        with open(file_path, "wb") as file, tqdm(
            desc=file_path,
            total=total_size,
            unit="iB",
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(block_size):
                file.write(data)
                bar.update(len(data))
        print(f"Download of {file_path} complete.")


def extract_wordnet16_tar(tar_path, extract_path):
    def is_within_directory(directory, target):
        abs_directory = os.path.abspath(directory)
        abs_target = os.path.abspath(target)
        return os.path.commonpath([abs_directory]) == os.path.commonpath(
            [abs_directory, abs_target]
        )

    def safe_extract(tar, path=".", members=None, *, numeric_owner=False):
        for member in tar.getmembers():
            member_path = os.path.join(path, member.name)
            if not is_within_directory(path, member_path):
                raise Exception("Attempted Path Traversal in Tar File")
        for member in members:
            if member.issym() or member.islnk():
                continue  # Skip symbolic links
            tar.extract(member, path, numeric_owner=numeric_owner, filter=None)

    with tarfile.open(tar_path, "r") as tar:
        members = [m for m in tar.getmembers() if m.name.startswith("wordnet-1.6/")]
        for member in members:
            member.name = member.name[len("wordnet-1.6/") :]
        safe_extract(tar, extract_path, members=members)


def download_and_extract_targz(url, gz, path):
    if os.path.exists(path):
        print(f"File {path} already exists.")
    else:
        print(f"Downloading and extracting the {gz} to {path}.")
        download(url, gz)
        extract_wordnet16_tar(gz, path)
        os.remove(gz)

Download the Finnish Wikipedia 2017 dataset, Finnish Stopwords, Stemmer, FinnWordNet and Word2Vec.


In [ ]:
# Wikipedia dataset
wiki_url = "https://www.kielipankki.fi/download/wikipedia-fi/wikipedia-fi-2017-src/wikipedia-fi-2017-src.zip"
wiki_zip_path = "wikipedia-fi-2017-src.zip"
wiki_extract_path = "data/wikipedia-fi-2017-src"
download_and_extract(wiki_url, wiki_zip_path, wiki_extract_path)

# Finnish Stopwords
stopword_url = "http://members.unine.ch/jacques.savoy/clef/finnishST.txt"
stopwords_path = "data/finnishST.txt"
download(stopword_url, stopwords_path)

# Finnish Stemmer
# stemmer_url = "http://members.unine.ch/jacques.savoy/clef/finnishStemmer.txt"
# stemmer_path = "data/finnishStemmer.txt"
# download(stemmer_url, stemmer_path)

# FinnWordNet
finnwordnet_url = (
    "https://www.kielipankki.fi/download/FinnWordNet/v2.0/FinnWordNet-2.0.zip"
)
finnwordnet_zip_path = "FinnWordNet-2.0.zip"
finnwordnet_extract_path = "data/FinnWordNet"
download_and_extract(finnwordnet_url, finnwordnet_zip_path, finnwordnet_extract_path)

# Wordnet 1.6
wordnet16_url = "https://wordnetcode.princeton.edu/1.6/wn16.unix.tar.gz"
wordnet16_gz = "wn16.unix.tar.gz"
wordnet16_path = "data/wn16"
download_and_extract_targz(wordnet16_url, wordnet16_gz, wordnet16_path)

# Download Wordnet Affect
wordnet_affect_url = "https://unioulu-my.sharepoint.com/:u:/g/personal/jyrjanai20_student_oulu_fi/EcfjOMVVQ7dIr_lETcSOFo4B5KEtVvB1I5mmWEVxjspDNQ?e=hKkcAj&download=1"
wordnet_affect_zip = "wndomains32.zip"
wordnet_affect_path = "data/wndomains32"
download_and_extract(wordnet_affect_url, wordnet_affect_zip, wordnet_affect_path)

In [7]:
# !python -m spacy download fi_core_news_md
# !python -m spacy download fi_core_news_sm

### Task 1:

Consider the wordings: “climate change”, “emission”, “resilience”, “sustainability” (need to consider their Finnish translations). Suggest a script where your input each of these wordings and output the corresponding Wikipedia pages, highlighting all the (linked) entities in these pages.


In [8]:
terms = {"ilmastonmuutos", "päästö", "joustavuus", "ekologinen kestävyys"}

In [9]:
# Function to load the extracted Wikipedia dataset
def load_wikipedia_dataset(directory):
    data = {}
    data_titles = {}

    # Iterate over all dataset parts
    files = [
        f
        for f in os.listdir(directory)
        if f.startswith("wiki_part") and f.endswith(".VRT")
    ]
    for filename in tqdm(files, desc="Processing files"):
        if filename.startswith("wiki_part") and filename.endswith(".VRT"):
            file_path = os.path.join(directory, filename)
            # print(f"Processing file: {file_path}")

            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()

                # Use regex to find all documents within the <doc>...</doc> tags
                docs = re.findall(
                    r'<doc id="(.*?)" url="(.*?)" title="(.*?)">(.*?)</doc>',
                    content,
                    re.DOTALL,
                )

                # Process each document found
                for doc_id, url, title, doc_content in docs:
                    data_titles[title.casefold()] = {
                        "id": doc_id,
                    }
                    # Only store the data if the title matches one of the specified terms
                    if title.casefold() in terms:
                        data[title.casefold()] = {
                            "id": doc_id,
                            "url": url,
                            "content": doc_content,
                        }
    return data, data_titles


# Load Finnish stopwords
def load_stopwords(file_path):
    with open(file_path, "r", encoding="ISO-8859-1") as f:
        stopwords = f.read().splitlines()
        print(f"Loaded {len(stopwords)} stopwords from {file_path}")
    return stopwords


# Function to highlight linked entities
def highlight_linked_entities(content):
    highlighted_content = re.sub(
        r'(<link entity=")(.*?)(">)(.*?)(</link>)', r"\1\2\3**\4**\5", content
    )
    return highlighted_content


# Function to tokenize text into sentences and remove stopwords
def remove_stopwords(sentence, stopwords, language="finnish"):
    words = word_tokenize(text=sentence, language=language)
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return " ".join(filtered_words)


# Function to remove '#' from the middle of a word
def remove_hash_from_words(text):
    return re.sub(r"(\w)#(\w)", r"\1\2", text)

Load the wikipedia dataset


In [ ]:
dataset_directory = "data/wikipedia-fi-2017-src"  # Path to the extracted dataset
wikipedia_data, wikipedia_title_data = load_wikipedia_dataset(dataset_directory)
print(f"number of wikipedia titles", len(wikipedia_title_data))

Load the finnish stopwords


In [ ]:
finnish_stopwords = load_stopwords(stopwords_path)

Highlight the entities in the wikipedia pages


In [12]:
for title, info in wikipedia_data.items():
    highlighted_content = highlight_linked_entities(info["content"])
    # print(f"Title: {title}\nHighlighted Content: {highlighted_content}\n")

Get the pages for the given wordings and get the sentences


In [ ]:
# Function to extract third column data from sentences
def extract_third_column(sentence):
    third_column_data = []
    lines = sentence.strip().split("\n")
    for line in lines:
        if line.strip() and not line.startswith("<"):
            columns = line.split("\t")
            if len(columns) > 2:
                third_column_data.append(columns[2])
    return third_column_data


def process_paragraphs(input_text):
    # print(f"Input Text: {input_text}")
    paragraphs = re.findall(r"<paragraph>(.*?)</paragraph>", input_text, re.DOTALL)
    all_third_column_data = []
    # print(f"Paragraphs: {paragraphs}")
    for paragraph in paragraphs:
        sentences = re.findall(r"<sentence>(.*?)</sentence>", paragraph, re.DOTALL)
        # print(f"Sentences: {sentences}")
        for sentence in sentences:
            # print(f"Sentence: {sentence}")
            third_column_data = extract_third_column(sentence)
            all_third_column_data.append(" ".join(third_column_data))
    return all_third_column_data


# print(f"Wikipedia data: {wikipedia_data['ilmastonmuutos']['content']}")

# Process each item in wikipedia_data
for item in wikipedia_data.items():
    third_column_data = process_paragraphs(item[1]["content"])
    # print(f"Item {item[0]} third column data:")
    for j, data in enumerate(third_column_data):
        filtered_sentence = remove_stopwords(data, finnish_stopwords)
        # print(f"Sentence {j+1}: {filtered_sentence}")
        # Save the filtered sentence to a variable if needed
        # For example: sentence_var = filtered_sentence
    # print("\n" + "#" * 80 + "\n")


all_words = {}

print(wikipedia_data)
for title, document in wikipedia_data.items():
    sentences = re.findall(
        r"<sentence>(.*?)</sentence>", document["content"], re.DOTALL
    )
    document_words = []
    if sentences:
        for idx, sentence_content in enumerate(sentences):
            # print(f"Processing sentence {idx + 1}...")

            # Split each sentence into individual lines
            lines = sentence_content.strip().split("\n")

            # cleaned_lines = re.sub(r'[^\w\s]', '', lines)
            # Extract the word (3nd column) from each line
            words = [line.split("\t")[2] for line in lines if line.strip()]
            for word in words:
                cleaned_word = re.sub(r"[^\w\s]", "", word)
                if not cleaned_word or cleaned_word in finnish_stopwords:
                    continue
                document_words.append(cleaned_word.casefold())

    all_words[title] = document_words

for title, words in all_words.items():
    print(f"\nWords for document '{title}': {words}")

In [ ]:
entities_found = {}

for i, topic_words in enumerate(all_words):
    found_words = []
    for word in all_words[topic_words]:
        if word.casefold() in [title.casefold() for title in wikipedia_title_data]:
            found_words.append(word.casefold())
    entities_found[topic_words] = found_words


# Example usage of the entities_found dictionary
for topic, found_entities in entities_found.items():
    print(f"Document '{topic}' contains the following entities: {found_entities}")

### Task 2

Assume the content of each webpage is a single document. Use relevant NLTK script to create a corpus constituted of the four document, and appropriate proprocessing and lemmatization, to construct the TfIdfVectorizer of each document and then calculate the cosine similarity of each pair of these documents. Provide the result in a table and comment on the findings.


In [15]:
# def extract_second_column(sentence):
#     second_column_data = []
#     lines = sentence.strip().split("\n")
#     for line in lines:
#         if line.strip() and not line.startswith("<"):
#             columns = line.split("\t")
#             if len(columns) > 2:
#                 second_column_data.append(columns[1])
#     return second_column_data


# def process_sentences(input_text):
#     paragraphs = re.findall(r"<paragraph>(.*?)</paragraph>", input_text, re.DOTALL)
#     all_second_column_data = []
#     for paragraph in paragraphs:
#         sentences = re.findall(r"<sentence>(.*?)</sentence>", paragraph, re.DOTALL)
#         # print(f"Sentences: {sentences}")
#         for sentence in sentences:
#             # print(f"Sentence: {sentence}")
#             second_column_data = extract_second_column(sentence)
#             all_second_column_data.append(" ".join(second_column_data))
#     return all_second_column_data


# Preprocessing function to remove stopwords, stemming, and tokenize the document
def preProcess(words, stopwords):
    tokens = []
    clean_stopwords = [word.casefold() for word in stopwords]
    stemmer = SnowballStemmer("finnish")
    # sentences = process_sentences(doc)
    # print(f"Sentences: {sentences}")
    # sentences = sent_tokenize(text=sentences.casefold(), language='finnish')

    # words = word_tokenize(words)

    words = [stemmer.stem(word) for word in words]
    words = [word for word in words if word.isalpha() and word not in clean_stopwords]
    words_set = list(set(words))

    tokens.extend(words_set)

    return " ".join(tokens)

In [ ]:
# Preprocess the document
corpus = []

for title, words in all_words.items():
    processed_doc = preProcess(words, finnish_stopwords)
    print(f"Processed Document: {processed_doc}")
    corpus.append(processed_doc)

# Print the processed document
print("\nCorpus:")
for doc in corpus:
    print(doc)

In [17]:
# TF-IDF Vectorization of each document
tf = TfidfVectorizer(use_idf=True, min_df=1)
tfidf_matrix = tf.fit_transform(corpus)

In [18]:
# Compute cosine similarity of each pair of these documents
cosine_sim = cosine_similarity(tfidf_matrix)

In [ ]:
# Provide the result in a table
cosine_sim_df = pd.DataFrame(
    cosine_sim,
    index=[f"Doc{i+1}" for i in range(len(cosine_sim))],
    columns=[f"Doc{i+1}" for i in range(len(cosine_sim))],
)

print(cosine_sim_df)

# Comment on the findings

### Task 3

Repeat 2) when the documents are restricted only to clickable entities of each document


In [20]:
# Preprocessing function to remove stopwords, stemming
def preProcess_entites(words, stopwords):
    clean_stopwords = [word.casefold() for word in stopwords]
    stemmer = SnowballStemmer("finnish")

    words = [stemmer.stem(word) for word in words]

    words = [word for word in words if word.isalpha() and word not in clean_stopwords]

    words = list(set(words))

    return " ".join(words)

In [ ]:
# Preprocess the document
corpus_entity = []
for document, found_entities in entities_found.items():
    processed_doc = preProcess_entites(found_entities, finnish_stopwords)
    corpus_entity.append(processed_doc)

# Print the processed document
print("\nCorpus_entity:")
for doc in corpus_entity:
    print(doc)

In [22]:
# TF-IDF Vectorization of each document
tf = TfidfVectorizer(use_idf=True, min_df=1)
tfidf_matrix_entity = tf.fit_transform(corpus_entity)

In [23]:
# Compute cosine similarity of each pair of these documents
cosine_sim_entity = cosine_similarity(tfidf_matrix_entity)

In [ ]:
# Provide the result in a table
cosine_sim_df_entity = pd.DataFrame(
    cosine_sim_entity,
    index=[f"Doc{i+1}" for i in range(len(cosine_sim_entity))],
    columns=[f"Doc{i+1}" for i in range(len(cosine_sim_entity))],
)

print(cosine_sim_df_entity)

# Comment on the findings

### Task 4

Write a script that explores the clickable entities in each of the four concepts (exploring only once), and generates new extended vocabulary for each concepts, and compute the new similarity between each pair of concepts using extent of overlapping of overall vocabulary and reduced vocabulary (when restricting the vocabulary to clickable entities). We shall refer to the case where the clickable entities are further explored as extended vocabulary case.


In [25]:
# Function to further explore clickable entities and generate extended vocabulary
# Function to load the extracted Wikipedia dataset
def load_wikipedia_dataset_extended(directory):
    data_entity_voc = {}
    nodup = {}
    categories = ["ilmastonmuutos", "päästö", "joustavuus", "ekologinen kestävyys"]
    data_entity_voc = {category: [] for category in categories}
    for topic, found_entities in entities_found.items():
        nodup[topic] = list(set(found_entities))

    # Iterate over all dataset parts
    files = [
        f
        for f in os.listdir(directory)
        if f.startswith("wiki_part") and f.endswith(".VRT")
    ]

    for filename in tqdm(files, desc="Processing files"):
        if filename.startswith("wiki_part") and filename.endswith(".VRT"):
            file_path = os.path.join(directory, filename)
            # print(f"Processing file: {file_path}")

            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()

                # Use regex to find all documents within the <doc>...</doc> tags
                docs = re.findall(
                    r'<doc id="(.*?)" url="(.*?)" title="(.*?)">(.*?)</doc>',
                    content,
                    re.DOTALL,
                )

                # Process each document found
                for doc_id, url, title, doc_content in docs:
                    # Loop through the relevant categories in 'nodup' to reduce repetitive code
                    for category in categories:
                        # Check if the title matches the category terms in 'nodup'
                        if title.casefold() in nodup[category]:
                            # Create a dictionary with the document data
                            document_data = {
                                "title": title,
                                "id": doc_id,
                                "url": url,
                                "content": doc_content,
                            }

                            # Append the document data to the corresponding category in 'data_entity_voc'
                            data_entity_voc[category].append(document_data)

    return data_entity_voc

In [ ]:
#
dataset_directory = "data/wikipedia-fi-2017-src"  # Path to the extracted dataset
documents_extended = load_wikipedia_dataset_extended(dataset_directory)

In [27]:
def list_extended_words(documents_extended):
    all_words_extended = {}

    for title, documents in documents_extended.items():
        for doc in documents:
            sentences = re.findall(
                r"<sentence>(.*?)</sentence>", doc["content"], re.DOTALL
            )
            document_words = []
            if sentences:
                for idx, sentence_content in enumerate(sentences):
                    # print(f"Processing sentence {idx + 1}...")

                    # Split each sentence into individual lines
                    lines = sentence_content.strip().split("\n")

                    # cleaned_lines = re.sub(r'[^\w\s]', '', lines)
                    # Extract the word (3nd column) from each line
                    words = [line.split("\t")[2] for line in lines if line.strip()]
                    for word in words:
                        cleaned_word = re.sub(r"[^\w\s]", "", word)
                        if not cleaned_word or cleaned_word in finnish_stopwords:
                            continue
                        document_words.append(cleaned_word.casefold())

            if title in all_words_extended:
                all_words_extended[title].extend(document_words)
            else:
                # If the title doesn't exist, initialize it with the document_words list
                all_words_extended[title] = document_words

    return all_words_extended

In [28]:
words_extended = list_extended_words(documents_extended)

words_extended_all = (
    {}
)  # extended words for all documents "ilmastonmuutos", "päästö", "joustavuus", "ekologinen kestävyys"

categories = ["ilmastonmuutos", "päästö", "joustavuus", "ekologinen kestävyys"]
words_extended_all = {category: [] for category in categories}

for title, words in all_words.items():
    words_extended_all[title] = all_words[title] + words_extended[title]

In [29]:
# Preprocessing function to remove stopwords, stemming
def preProcess_extended_words(words, stopwords):
    clean_stopwords = [word.casefold() for word in stopwords]
    stemmer = SnowballStemmer("finnish")

    words = [stemmer.stem(word) for word in words]

    words = [word for word in words if word.isalpha() and word not in clean_stopwords]

    words = list(set(words))

    return " ".join(words)


# Compute similiarity for the extended words


def compute_similarity(vocab1, vocab2):
    # Join vocabularies into a single string for each concept
    text1 = " ".join(vocab1)
    text2 = " ".join(vocab2)

    # Use TfidfVectorizer to create a matrix of the text
    tf = TfidfVectorizer(use_idf=True, min_df=1)
    tfidf_matrix = tf.fit_transform([text1, text2])

    # Compute cosine similarit
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

    return cosine_sim

In [30]:
# Preprocess the extended words + original words and extended words

words_extended_preproces = {}
words_extended_all_preproces = {}

categories = ["ilmastonmuutos", "päästö", "joustavuus", "ekologinen kestävyys"]
words_extended_preproces = {category: [] for category in categories}
words_extended_all_preproces = {category: [] for category in categories}

for document, words in words_extended.items():
    processed_doc_ext = preProcess_entites(words, finnish_stopwords)
    words_extended_preproces[document].append(processed_doc_ext)

for document, words in words_extended_all.items():
    processed_doc_ext_all = preProcess_entites(words, finnish_stopwords)
    words_extended_all_preproces[document].append(processed_doc_ext_all)

In [ ]:
# Compute similarity for the extended words + original words and extended words

similarities = {}
for title in categories:
    similarities[title] = compute_similarity(
        words_extended_preproces[title], words_extended_all_preproces[title]
    )

for title, similarity in similarities.items():
    print(
        f"Similarity between the extended vocabulary of '{title}' and the original vocabulary: {similarity}"
    )

### Task 5

We want to assess the importance of each of the four concepts by the number of clickable entities including in the webpage of the given concept. Write a script that implements this strategy to determine the importance of each concept. Next, we want to use the extended vocabulary by quantifying the importance of individual entity in the original webpage by the number of clickable links it generates. Summarize a table highlighting the order of importance of the four concepts according to each of the original and extended vocabulary.


In [ ]:
# calculate the occurance of each entity in the entities_found
entity_occurence = {}
for topic, found_entities in entities_found.items():
    entity_occurence[topic] = {}
    for entity in found_entities:
        if entity in entity_occurence[topic]:
            entity_occurence[topic][entity] += 1
        else:
            entity_occurence[topic][entity] = 1

# order the entities by occurance
sorted_entities = {}
for topic, entities in entity_occurence.items():
    sorted_entities[topic] = {
        k: v
        for k, v in sorted(entities.items(), key=lambda item: item[1], reverse=True)
    }

## print the sorted entities for each topic
for topic, entities in sorted_entities.items():
    print(f"Document '{topic}' contains entities:")
    print(entities)

In [ ]:
## words_extended_all contains the extended vocabulary for each document
## entities_found contains the entities found in each document


def count_clickable_links(entities):
    return {entity: len(entity) for entity in entities}


# Count clickable links for original and extended vocabulary
original_counts = {
    concept: count_clickable_links(entities)
    for concept, entities in entities_found.items()
}
extended_counts = {
    concept: count_clickable_links(entities)
    for concept, entities in words_extended_all.items()
}

# Summarize importance
original_importance = {
    concept: sum(counts.values()) for concept, counts in original_counts.items()
}
extended_importance = {
    concept: sum(counts.values()) for concept, counts in extended_counts.items()
}

# Create a DataFrame to summarize the importance
importance_df = pd.DataFrame(
    {
        "Concept": list(original_importance.keys()),
        "Original Importance": list(original_importance.values()),
        "Extended Importance": list(extended_importance.values()),
    }
)

# Sort the DataFrame by importance
importance_df = importance_df.sort_values(
    by=["Original Importance", "Extended Importance"], ascending=False
)

# Display the table
print(importance_df)

### Task 6

We want to assess the similarity between the concepts is reflected in the their webpage content. Use a script to calculate Wu and Palmer WordNet semantic similarity between each pair of the four concepts and then compare result with the Jaccard similarity obtained by both original vocabulary in 2) and extended vocabulary 4), and comment on the compatibility between the semantic similarity and the above constructed Jaccard similarity measures


In [ ]:
finnwordnet_path = "data/FinnWordNet/dict"
# wn = WordNetCorpusReader(finnwordnet_path, None)

fi = wn.Wordnet("omw-fi")

In [ ]:
# Wu and Palmer WordNet Semantic Similarity Calculation
terms_list = list(terms)
wu_palmer_similarity = []
for i in range(len(terms_list)):
    row = []
    for j in range(len(terms_list)):
        if i == j:
            row.append(1.0)
        else:
            synsets_1 = fi.synsets(terms_list[i])
            synsets_2 = fi.synsets(terms_list[j])
            max_similarity = 0
            for syn1 in synsets_1:
                for syn2 in synsets_2:
                    similarity = wup(syn1, syn2, simulate_root=True)
                    if similarity and similarity > max_similarity:
                        max_similarity = similarity
            row.append(max_similarity if max_similarity else 0)
    wu_palmer_similarity.append(row)

# Provide the result in a table
wu_palmer_sim_df = pd.DataFrame(
    wu_palmer_similarity,
    index=[f"Concept{i+1}" for i in range(len(terms_list))],
    columns=[f"Concept{i+1}" for i in range(len(terms_list))],
)

print("\nWu and Palmer WordNet Semantic Similarity:")
print(wu_palmer_sim_df)


# Jaccard Similarity Calculation for Original and Extended Vocabulary
def jaccard_similarity(set1, set2):
    intersection = len(set(set1).intersection(set(set2)))
    union = len(set(set1).union(set(set2)))
    return intersection / union if union != 0 else 0


# Calculate Jaccard Similarity for Original and Extended Vocabularies
jaccard_original = []
jaccard_extended = []
for i in range(len(terms_list)):
    row_original = []
    row_extended = []
    for j in range(len(terms_list)):
        if i == j:
            row_original.append(1.0)
            row_extended.append(1.0)
        else:
            row_original.append(
                jaccard_similarity(all_words[terms_list[i]], all_words[terms_list[j]])
            )
            row_extended.append(
                jaccard_similarity(
                    words_extended_all[terms_list[i]], words_extended_all[terms_list[j]]
                )
            )
    jaccard_original.append(row_original)
    jaccard_extended.append(row_extended)

# Provide the results in tables
jaccard_original_df = pd.DataFrame(
    jaccard_original,
    index=[f"Concept{i+1}" for i in range(len(terms_list))],
    columns=[f"Concept{i+1}" for i in range(len(terms_list))],
)

jaccard_extended_df = pd.DataFrame(
    jaccard_extended,
    index=[f"Concept{i+1}" for i in range(len(terms_list))],
    columns=[f"Concept{i+1}" for i in range(len(terms_list))],
)

print("\nJaccard Similarity (Original Vocabulary):")
print(jaccard_original_df)
print("\nJaccard Similarity (Extended Vocabulary):")
print(jaccard_extended_df)

# Comment on Compatibility Between Similarity Measures
print("\nComments on Compatibility Between Semantic and Jaccard Similarities:")
for i in range(len(terms_list)):
    for j in range(i + 1, len(terms_list)):
        print(f"\nComparison between Concept{i+1} and Concept{j+1}:")
        print(f"Wu and Palmer Semantic Similarity: {wu_palmer_sim_df.iloc[i, j]:.2f}")
        print(
            f"Jaccard Similarity (Original Vocabulary): {jaccard_original_df.iloc[i, j]:.2f}"
        )
        print(
            f"Jaccard Similarity (Extended Vocabulary): {jaccard_extended_df.iloc[i, j]:.2f}"
        )
        if wu_palmer_sim_df.iloc[i, j] > 0.5:
            print(
                "High semantic similarity is generally compatible with high Jaccard similarity."
            )
        else:
            print(
                "Low semantic similarity indicates less compatibility with Jaccard similarity."
            )

### Task 7

Now we want to evaluate the emotion of the text in each dataframe. For this purpose, use WordnetAffect (https://github.com/clemtoy/WNAffect), which generates an emotion state and the part of speech tag of each word matched in the lexicon, and then compile the overall output for each concept. Repeat this process for both restricted and extended vocabulary cases, and comment on the compatibility between the emotion gained from the corresponding Wikipedia page and the intuitive emotion from individual concept.


In [36]:
# Emotional Damage          Analysis Using WordNetAffect
wna = WNAffect(wordnet16_path, wordnet_affect_path)
translator = GoogleTranslator(source="fi", target="en")

In [ ]:
def translate_word(word):
    """Translate a single word."""
    return translator.translate(word)


def translate_batch(words):
    """Translate a batch of words."""
    return translator.translate_batch(words)


def load_or_translate(all_words, filename="translated_all_words.pkl"):
    """Load translated words from disk if available, otherwise translate and save to disk."""
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            translated_all_words = pickle.load(f)
        print("Loaded translated words from disk.")
    else:
        translated_all_words = {}
        for term, vocab in all_words.items():
            translated_all_words[term] = translate_batch_marian(vocab)
            print(f"Translated words for '{term}'")

        # translated_all_words = {}
        # total_words = sum(len(vocab) for vocab in all_words.values())

        # with tqdm(total=total_words, desc="Translating words") as pbar:
        #     with concurrent.futures.ThreadPoolExecutor() as executor:
        #         future_to_word = {executor.submit(translate_word, word): (term, word) for term, vocab in all_words.items() for word in vocab}
        #         for future in concurrent.futures.as_completed(future_to_word):
        #             term, word = future_to_word[future]
        #             translated_word = future.result()
        #             if term not in translated_all_words:
        #                 translated_all_words[term] = []
        #             translated_all_words[term].append(translated_word)
        #             pbar.update(1)

        with open(filename, "wb") as f:
            pickle.dump(translated_all_words, f)
        print("Translated words saved to disk.")
    return translated_all_words


# use MarianMTModel for translation
def translate_batch_marian(words, model_name="Helsinki-NLP/opus-mt-fi-en"):
    print(f"CUDA: {torch.cuda.is_available()}")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")
    torch.cuda.empty_cache()
    model = MarianMTModel.from_pretrained(model_name).to(device)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    # Generate translation
    inputs = tokenizer(words, return_tensors="pt", padding=True).to(device)
    # Generate translation
    translated = model.generate(**inputs)

    # Decode the translated text
    translated_text = [
        tokenizer.decode(t, skip_special_tokens=True) for t in translated
    ]
    return translated_text


# # Load the model and tokenizer
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model_name = 'Helsinki-NLP/opus-mt-fi-en'
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name).to(device)

# # Translate from Finnish to English
# src_text = ["kissa"]  # Example text in Finnish

# # Generate translation
# inputs = tokenizer(src_text, return_tensors="pt", padding=True).to(device)

# # Generate translation
# translated = model.generate(**inputs)

# # Decode the translated text
# translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
# print(translated_text)

In [ ]:
print(f"CUDA: {torch.cuda.is_available()}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]


def translate_batch_marian(
    words, model_name="Helsinki-NLP/opus-mt-fi-en", batch_size=8
):
    torch.cuda.empty_cache()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MarianMTModel.from_pretrained(model_name).to(device)
    tokenizer = MarianTokenizer.from_pretrained(model_name)

    translations = []
    total_words = len(words)
    with tqdm(total=total_words, desc="Translating batches") as pbar:
        for src_text_list in chunks(words, batch_size):
            batch = tokenizer(
                src_text_list, return_tensors="pt", padding=True, truncation=True
            ).to(device)
            gen = model.generate(**batch)
            translated_text = tokenizer.batch_decode(gen, skip_special_tokens=True)
            translations.extend(translated_text)
            pbar.update(len(src_text_list))
            # print(f"Translated {translations} words")

    return translations


def load_or_translate(all_words, filename="translated_all_words.pkl", batch_size=128):
    """Load translated words from disk if available, otherwise translate and save to disk."""
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            translated_all_words = pickle.load(f)
        print("Loaded translated words from disk.")
    else:
        translated_all_words = {}
        for term, vocab in all_words.items():
            translated_vocab = translate_batch_marian(vocab, batch_size=batch_size)
            translated_all_words[term] = translated_vocab

        with open(filename, "wb") as f:
            pickle.dump(translated_all_words, f)
        print("Translated words saved to disk.")
    return translated_all_words


# Usage
translated_all_words = load_or_translate(all_words, "translated_all_words.pkl")
translated_extended_words = load_or_translate(
    words_extended_all, "translated_extended_words.pkl"
)

In [ ]:
print(translated_all_words)
print(translated_extended_words)
print(len(translated_extended_words))

In [ ]:
# Usage
translated_all_words = load_or_translate(all_words, "translated_all_words.pkl")
translated_extended_words = load_or_translate(
    words_extended_all, "translated_extended_words.pkl"
)

In [39]:
# Translate the terms in "words_extended_all" to English

# for term, vocab in words_extended_all.items():
# emotion_results_extended[term] = analyze_emotions_v2(" ".join(vocab))

In [40]:
# Function to analyze emotions of text using WordNetAffect
def analyze_emotions(text):
    words = text.split()
    emotion_count = defaultdict(int)
    for word in tqdm(words, desc="Processing words"):
        pos_tag = nltk.pos_tag([word])[0][1]
        synsets = wn.synsets(word, pos=pos_tag)
        for syn in synsets:
            emotion = wna.get_emotion(syn, pos_tag)
            if emotion:
                emotion_count[emotion] += 1
    return emotion_count


#######
#######
# def process_word(word):
#     """Process a single word to get its emotion."""
#     pos_tagged = nltk.pos_tag([word])
#     pos = nltk.pos_tag([word])[0][1]
#     synsets = wn.synsets(word, pos=pos)
#     emotion_count = defaultdict(int)
#     for syn in synsets:
#         emotion = wna.get_emotion(syn, pos_tagged[0][1])
#         if emotion:
#             emotion_count[emotion] += 1
#     return emotion_count

# def analyze_emotions_v2(text):
#     words = text.split()
#     emotion_count = defaultdict(int)
#     manager = Manager()
#     counter = manager.Value('i', 0)
#     total = len(words)
#     with Pool() as pool:
#         results = list(tqdm(pool.imap(process_word, [(word, counter, total) for word in words]), total=total, desc="Processing words"))
#     for result in results:
#         for emotion, count in result.items():
#             emotion_count[emotion] += count
#     return emotion_count
#######
#######

In [ ]:
#######
#######
####### USE MULTI-THREADING TO SPEED UP THE PROCESS

# # Analyze emotions for original and extended vocabulary
# emotion_results_original = {}
# emotion_results_extended = {}
# for term, vocab in translated_all_words.items():
#     emotion_results_original[term] = analyze_emotions_v2(" ".join(vocab))
# for term, vocab in translated_extended_words.items():
#     emotion_results_extended[term] = analyze_emotions_v2(" ".join(vocab))

# # Print emotion analysis results
# print("\nEmotion Analysis (Original Vocabulary):")
# for term, emotions in emotion_results_original.items():
#     print(f"{term}: {dict(emotions)}")

# print("\nEmotion Analysis (Extended Vocabulary):")
# for term, emotions in emotion_results_extended.items():
#     print(f"{term}: {dict(emotions)}")

# # Comment on Compatibility Between Emotion Analysis and Intuitive Emotion
# print("\nComments on Compatibility Between Emotion Analysis and Intuitive Emotion:")
# for term in terms_list:
#     print(f"\nConcept: {term}")
#     print(f"Emotion Analysis (Original Vocabulary): {emotion_results_original[term]}")
#     print(f"Emotion Analysis (Extended Vocabulary): {emotion_results_extended[term]}")
#     # Add your own intuitive analysis based on the concept meaning
#     print(
#         "The intuitive emotion for this concept aligns/does not align well with the emotion extracted from the Wikipedia page."
#     )

In [ ]:
# Analyze emotions for original and extended vocabulary
emotion_results_original = {}
emotion_results_extended = {}
for term, vocab in translated_all_words.items():
    emotion_results_original[term] = analyze_emotions(" ".join(vocab))
for term, vocab in translated_extended_words.items():
    emotion_results_extended[term] = analyze_emotions(" ".join(vocab))

# Print emotion analysis results
print("\nEmotion Analysis (Original Vocabulary):")
for term, emotions in emotion_results_original.items():
    print(f"{term}: {dict(emotions)}")

print("\nEmotion Analysis (Extended Vocabulary):")
for term, emotions in emotion_results_extended.items():
    print(f"{term}: {dict(emotions)}")

# Comment on Compatibility Between Emotion Analysis and Intuitive Emotion
print("\nComments on Compatibility Between Emotion Analysis and Intuitive Emotion:")
for term in terms_list:
    print(f"\nConcept: {term}")
    print(f"Emotion Analysis (Original Vocabulary): {emotion_results_original[term]}")
    print(f"Emotion Analysis (Extended Vocabulary): {emotion_results_extended[term]}")
    # Add your own intuitive analysis based on the concept meaning
    print(
        "The intuitive emotion for this concept aligns/does not align well with the emotion extracted from the Wikipedia page."
    )

### Task 8

We want to repeat this process for sentiment polarity. For this purpose, use the Vader sentiment https://github.com/cjhutto/vaderSentiment to assess the sentiment of each the Wikipedia page associated to each concept (both with original vocabulary and extended vocabulary, by aggregating the sentiment of individual clickable entity’s page) to compute the sentiment associated with each concept. Comment on the sentiment of each concept in restricted and extended vocabulary and its compatibility with the intuitive sentiment gained from the inherent definition of these concepts.


In [ ]:
# Initialize Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Sentiment Analysis Using Vader Sentiment
# Function to analyze sentiment of text using Vader Sentiment Analyzer
def analyze_sentiment(text):
    translated_text = translator.translate(text, src="fi", dest="en")
    sentiment = analyzer.polarity_scores(translated_text)
    return sentiment


# Analyze sentiment for original and extended vocabulary
sentiment_results_original = {}
sentiment_results_extended = {}
for term, vocab in all_words.items():
    sentiment_results_original[term] = analyze_sentiment(" ".join(vocab))
for term, vocab in words_extended_all.items():
    sentiment_results_extended[term] = analyze_sentiment(" ".join(vocab))

# Print sentiment analysis results
print("\nSentiment Analysis (Original Vocabulary):")
for term, sentiment in sentiment_results_original.items():
    print(f"{term}: {sentiment}")

print("\nSentiment Analysis (Extended Vocabulary):")
for term, sentiment in sentiment_results_extended.items():
    print(f"{term}: {sentiment}")

# Comment on Compatibility Between Sentiment Analysis and Intuitive Sentiment
print("\nComments on Compatibility Between Sentiment Analysis and Intuitive Sentiment:")
for term in terms_list:
    print(f"\nConcept: {term}")
    print(
        f"Sentiment Analysis (Original Vocabulary): {sentiment_results_original[term]}"
    )
    print(
        f"Sentiment Analysis (Extended Vocabulary): {sentiment_results_extended[term]}"
    )
    # Add your own intuitive analysis based on the concept meaning
    print(
        "The intuitive sentiment for this concept aligns/does not align well with the sentiment extracted from the Wikipedia page."
    )

### Task 9

We want to investigate the similarity using the embedding representation. For this purpose, write a program that uses word2vec embedding to evaluate

i) the consistency of the Wikipedia page by calculating the cosine similarity between the underlined concept and the average word2vec embedding of every words contained in the page

ii) the similarity between the various concepts by computing the cosine similarity between the aggregated embedding vector of the corresponding pages. Comment on the compatibility between webpage content and concept and similarity between the various concepts.


##### i)


In [ ]:
# Function to get Word2Vec embedding of a word if it exists in the model
def get_word_embedding(word, model):
    try:
        # print(f"get_word_embedding: {word}")
        return model[word]
    except KeyError:
        return None


# Function to get average embedding
def get_average_embedding(tokens, model):
    embeddings = [get_word_embedding(token, model) for token in tokens]
    # print(f"embeddings 1: {embeddings}")
    embeddings = [embedding for embedding in embeddings if embedding is not None]
    # print(f"embeddings 2: {embeddings}")
    if not embeddings:
        return None
    return np.mean(embeddings, axis=0)


# Function to calculate cosine similarity
def cosine_similarity_vectors(v1, v2):
    v1 = v1.reshape(1, -1)
    v2 = v2.reshape(1, -1)
    return cosine_similarity(v1, v2)[0][0]

In [ ]:
# word2vec_fi = kagglehub.dataset_download("lehtol/word2vec-fi")
def evaluate_page_consistency(page_title, model, stop_words):
    # Get the page content
    # page_content = load_wikipedia_dataset(dataset_directory, page_title)
    # page_content = page_content["content"]

    page_content = all_words[page_title]

    # Preprocess the page content
    page_tokens = preProcess(page_content, stop_words)
    page_tokens = word_tokenize(page_tokens)

    # Get the embedding of the concept
    stemmer = SnowballStemmer("finnish")
    word = stemmer.stem(page_title)

    concept_embedding = get_word_embedding(word.casefold(), model)
    if concept_embedding is None:
        print(f"The concept '{page_title}' is not in the Word2Vec model vocabulary.")
        return

    # Get the average embedding of the page content
    avg_embedding = get_average_embedding(page_tokens, model)
    if avg_embedding is None:
        print(
            f"No words in the Wikipedia page '{page_title}' have embeddings in the Word2Vec model."
        )
        return

    # Calculate cosine similarity
    similarity = cosine_similarity_vectors(concept_embedding, avg_embedding)
    print(
        f"Cosine similarity between the page '{page_title}' and average embedding of the page content is: {similarity}"
    )

In [ ]:
# load the word2vec model
fi_model_path = os.path.join(word2vec_fi, "word2vec-fi.bin")
word2vec_fi_model = gensim.models.KeyedVectors.load_word2vec_format(
    fi_model_path, binary=True
)

## Evaluate the consistency of the Wikipedia page content with the concept "ilmastonmuutos"
# evaluate_page_consistency(
#     "ilmastonmuutos", word2vec_fi_model, finnish_stopwords
# )

# evaluate_page_consistency("kestävyys", word2vec_fi_model, finnish_stopwords)

for title in terms:
    evaluate_page_consistency(title, word2vec_fi_model, finnish_stopwords)

##### ii)


In [ ]:
# Function to get the aggregated embedding of a Wikipedia page
def get_page_embedding(page_title, model, stop_words):
    # Get the page content
    # page_content = load_wikipedia_dataset(dataset_directory, page_title)
    # page_content = page_content["content"]

    page_content = all_words[page_title]

    # Check if the page exists
    if page_content is None:
        print(f"The Wikipedia page does not exist.")
        return None

    # Preprocess the page content
    page_tokens = preProcess(page_content, stop_words)
    page_tokens = word_tokenize(page_tokens)

    # Get the average embedding of the page content
    avg_page_embedding = get_average_embedding(page_tokens, model)

    return avg_page_embedding


def cosine_similarity_between_pages(page_title_1, page_title_2, model, stop_words):
    # Get the aggregated embeddings for both pages
    # print(f"Calculating cosine similarity between '{page_title_1}' and '{page_title_2}'")
    embedding_1 = get_page_embedding(page_title_1, model, stop_words)
    embedding_2 = get_page_embedding(page_title_2, model, stop_words)

    # print(f"Embedding 1: {embedding_1}, Embedding 2: {embedding_2}")

    # Check if embeddings exist for both pages
    if embedding_1 is None or embedding_2 is None:
        print("One or both pages do not have valid embeddings.")
        return None

    # Reshape the embeddings for cosine similarity calculation
    embedding_1 = embedding_1.reshape(1, -1)
    embedding_2 = embedding_2.reshape(1, -1)

    # Calculate and return the cosine similarity
    similarity = cosine_similarity(embedding_1, embedding_2)[0][0]
    return similarity

In [ ]:
terms_list = list(terms)
# Calculate the cosine similarity between the pages and print the results in a matrix
page_similarity_matrix = np.zeros((len(terms_list), len(terms_list)))
for i, page_title_1 in enumerate(terms_list):
    for j, page_title_2 in enumerate(terms_list):
        similarity = cosine_similarity_between_pages(
            page_title_1, page_title_2, word2vec_fi_model, finnish_stopwords
        )
        page_similarity_matrix[i, j] = similarity

# Provide the result in a table
page_similarity_df = pd.DataFrame(
    page_similarity_matrix,
    index=terms_list,
    columns=terms_list,
)

print("\nCosine Similarity Between Wikipedia Pages:")
print(page_similarity_df)

# print results in a graph
plt.figure(figsize=(10, 8))
sns.heatmap(page_similarity_df, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Cosine Similarity Between Wikipedia Pages")
plt.xlabel("Wikipedia Page")
plt.ylabel("Wikipedia Page")
plt.show()

### Task 10

Repeat 9) when the extended vocabulary is considered.


##### i)


In [ ]:
# word2vec_fi = kagglehub.dataset_download("lehtol/word2vec-fi")
def evaluate_page_consistency(page_title, model, stop_words):
    # Get the page content
    page_content = words_extended_all[page_title]

    # Preprocess the page content
    clean_stopwords = [word.casefold() for word in stop_words]
    stemmer = SnowballStemmer("finnish")

    page_tokens = [stemmer.stem(word) for word in page_content]

    page_tokens = [
        word for word in page_tokens if word.isalpha() and word not in clean_stopwords
    ]

    page_tokens = list(set(page_tokens))

    # Get the embedding of the concept
    word = stemmer.stem(page_title)

    concept_embedding = get_word_embedding(word.casefold(), model)
    if concept_embedding is None:
        print(f"The concept '{page_title}' is not in the Word2Vec model vocabulary.")
        return

    # Get the average embedding of the page content
    avg_embedding = get_average_embedding(page_tokens, model)
    if avg_embedding is None:
        print(
            f"No words in the Wikipedia page '{page_title}' have embeddings in the Word2Vec model."
        )
        return

    # Calculate cosine similarity
    similarity = cosine_similarity_vectors(concept_embedding, avg_embedding)
    print(
        f"Cosine similarity between the page '{page_title}' and average embedding of the page content is: {similarity}"
    )

In [ ]:
for title in terms:
    evaluate_page_consistency(title, word2vec_fi_model, finnish_stopwords)

##### ii)


In [ ]:
# Function to get the aggregated embedding of a Wikipedia page
def get_page_embedding(page_title, model, stop_words):
    # Get the page content

    page_content = words_extended_all[page_title]

    # Check if the page exists
    if page_content is None:
        print(f"The Wikipedia page does not exist.")
        return None

    # Preprocess the page content
    clean_stopwords = [word.casefold() for word in stop_words]
    stemmer = SnowballStemmer("finnish")

    page_tokens = [stemmer.stem(word) for word in page_content]

    page_tokens = [
        word for word in page_tokens if word.isalpha() and word not in clean_stopwords
    ]

    page_tokens = list(set(page_tokens))

    # Get the average embedding of the page content
    avg_page_embedding = get_average_embedding(page_tokens, model)

    return avg_page_embedding


def cosine_similarity_between_pages(page_title_1, page_title_2, model, stop_words):
    # Get the aggregated embeddings for both pages
    # print(f"Calculating cosine similarity between '{page_title_1}' and '{page_title_2}'")
    embedding_1 = get_page_embedding(page_title_1, model, stop_words)
    embedding_2 = get_page_embedding(page_title_2, model, stop_words)

    # print(f"Embedding 1: {embedding_1}, Embedding 2: {embedding_2}")

    # Check if embeddings exist for both pages
    if embedding_1 is None or embedding_2 is None:
        print("One or both pages do not have valid embeddings.")
        return None

    # Reshape the embeddings for cosine similarity calculation
    embedding_1 = embedding_1.reshape(1, -1)
    embedding_2 = embedding_2.reshape(1, -1)

    # Calculate and return the cosine similarity
    similarity = cosine_similarity(embedding_1, embedding_2)[0][0]
    return similarity

In [ ]:
terms_list = list(terms)
# Calculate the cosine similarity between the pages and print the results in a matrix
page_similarity_matrix = np.zeros((len(terms_list), len(terms_list)))
for i, page_title_1 in enumerate(terms_list):
    for j, page_title_2 in enumerate(terms_list):
        similarity = cosine_similarity_between_pages(
            page_title_1, page_title_2, word2vec_fi_model, finnish_stopwords
        )
        page_similarity_matrix[i, j] = similarity

# Provide the result in a table
page_similarity_df = pd.DataFrame(
    page_similarity_matrix,
    index=terms_list,
    columns=terms_list,
)

print("\nCosine Similarity Between Wikipedia Pages:")
print(page_similarity_df)

# print results in a graph
plt.figure(figsize=(10, 8))
sns.heatmap(page_similarity_df, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Cosine Similarity Between Wikipedia Pages")
plt.xlabel("Wikipedia Page")
plt.ylabel("Wikipedia Page")
plt.show()

### Task 11

Use appropriate literature to comment on the findings. Also, identify any additional input that would allow you to further elucidate any of the preceding, and use appropriate literature of corpus linguistic literature to justify your findings and comment on the obtained results. Finally, comment on the limitations and structural weakness of the data processing pipeline.
